In [ ]:

import torch
import os
import random
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Set the seed for reproducibility
SEED = 123
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)  # Apply the seed


In [ ]:

BATCH_SIZE = 64
EPOCHS = 100
INPUT_SIZE = (256, 256)  

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

main_data_dir = r"C:\Users\Josh\Desktop\CUDA\skindiseases"
train_dir = os.path.join(main_data_dir, "train")
val_dir = os.path.join(main_data_dir, "val")
test_dir = os.path.join(main_data_dir, "test")

train_files = count_files_in_directory(train_dir)
val_files = count_files_in_directory(val_dir)
test_files = count_files_in_directory(test_dir)

print(f"Training Dataset: {train_files}")
print(f"Validation Dataset: {val_files}")
print(f"Test Dataset: {test_files}")

In [ ]:

# Load the training dataset to calculate mean and std, and get class labels
train_dataset = datasets.ImageFolder(root=train_dir)
class_n = list(train_dataset.class_to_idx.keys())  # Automatically retrieves class names from folders
print("Class to label mapping:", train_dataset.class_to_idx)

In [ ]:
# # Function to calculate mean and std for the dataset
# def calculate_mean_std(loader):
#     mean = 0.0
#     std = 0.0
#     total_images_count = 0
#     for images, _ in loader:
#         batch_samples = images.size(0)  # batch size (the last batch can have smaller size!)
#         images = images.view(batch_samples, images.size(1), -1)  # reshape to (batch_size, channels, height * width)
#         mean += images.mean(2).sum(0)
#         std += images.std(2).sum(0)
#         total_images_count += batch_samples

#     mean /= total_images_count
#     std /= total_images_count
#     return mean, std

# # Temporary transform to load the dataset without normalization for mean and std calculation
# transform_temp = transforms.Compose([
#     transforms.Resize(INPUT_SIZE),
#     transforms.CenterCrop((227, 227)),
#     transforms.ToTensor()
# ])

# # Load the training dataset without normalization
# train_dataset_temp = datasets.ImageFolder(root=train_dir, transform=transform_temp)
# train_loader_temp = DataLoader(train_dataset_temp, batch_size=BATCH_SIZE, shuffle=False)

# # Calculate mean and std
# mean, std = calculate_mean_std(train_loader_temp)
# print(f"Calculated Mean: {mean}")
# print(f"Calculated Std: {std}")

In [ ]:
import cv2
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans

class KMeansSegmentation:
    def __init__(self, n_clusters=3, overlay_alpha=0.5):
        self.n_clusters = n_clusters
        self.overlay_alpha = overlay_alpha

    def __call__(self, img):
        img_np = np.array(img)
        img_flat = img_np.reshape((-1, 3))
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=SEED).fit(img_flat)
        segmented_img = kmeans.cluster_centers_[kmeans.labels_].reshape(img_np.shape).astype(np.uint8)
        blended_img = cv2.addWeighted(img_np, 1 - self.overlay_alpha, segmented_img, self.overlay_alpha, 0)
        return Image.fromarray(blended_img)

In [ ]:
class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def __call__(self, img):
        img_np = np.array(img)

        img_lab = cv2.cvtColor(img_np, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(img_lab)

        clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
        l_clahe = clahe.apply(l)

        img_clahe = cv2.merge((l_clahe, a, b))
        img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_LAB2RGB)

        return Image.fromarray(img_clahe)

In [ ]:
import numpy as np
from torchvision import transforms

MEAN = (0.6181, 0.4643, 0.4194)
STD = (0.1927, 0.1677, 0.1617)


# Define transformations
transform_train = transforms.Compose([
    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.RandomResizedCrop(227, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5), 
    transforms.RandomApply([KMeansSegmentation(n_clusters=3, overlay_alpha=0.5)], p=0.3),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-20, 20))], p=0.5),
    transforms.RandomChoice([
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
        transforms.RandomGrayscale(p=0.2)
    ]),
    transforms.RandomApply([transforms.RandomAffine(degrees=5, translate=(0.02, 0.02), shear=2)], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

transform_val_test = transforms.Compose([

    CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8)),
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop((227,227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

# Load the datasets with the new transforms
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val_test)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_val_test)

# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
images, labels = next(iter(train_loader))
print(images.shape)  

In [ ]:

# Function to unnormalize the image for visualization
def unnormalize(image, mean, std):
    image = image.numpy().transpose((1, 2, 0))  
    image = (image * std) + mean  
    image = np.clip(image, 0, 1)  
    return image


# Visualize a batch of images from the train_loader
def visualize_loader(loader, mean, std, class_names, num_images=6):
    data_iter = iter(loader)
    images, labels = next(data_iter)  

    plt.figure(figsize=(12, 8))
    for i in range(num_images):
        plt.subplot(2, 3, i+1)
        image = unnormalize(images[i], mean, std)  
        plt.imshow(image)
        plt.title(f"Class: {class_names[labels[i]]}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

visualize_loader(train_loader, mean=MEAN, std=STD, class_names=class_n)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchsummary import summary
from torchvision.models import AlexNet_Weights

# Define the model
num_classes = 8

class AlexNet(nn.Module):
    def __init__(self, num_classes=num_classes):  
        super(AlexNet, self).__init__()
        
        self.model = models.alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
        self.model.classifier[6] = nn.Linear(4096, num_classes)  

    def forward(self, x):
        return self.model(x)

alexnet = AlexNet(num_classes=num_classes)
device = torch.device("cuda") 
alexnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=2e-4, weight_decay=1e-4) 

summary(alexnet, input_size=(3, 227, 227))
print(alexnet)


In [ ]:
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler

PATIENCE = 10 
MODEL = 'alexnet.pth'

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=PATIENCE)

# Training Loop
def train_model(model, train_loader, val_loader, epochs, criterion, optimizer, device, scheduler, patience, save_path=MODEL):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    best_val_accuracy = 0.0
    best_epoch = 0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch + 1}/{epochs}")
            for inputs, labels in tepoch:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                tepoch.set_postfix(loss=running_loss / len(train_loader), accuracy=100 * correct / total)

        epoch_train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total
        train_losses.append(epoch_train_loss)
        train_accuracies.append(train_accuracy)

        # Validation loop
        val_loss = 0.0
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            with tqdm(val_loader, unit="batch", leave=False) as vepoch:
                vepoch.set_description(f"Validation Epoch {epoch + 1}/{epochs}")
                for inputs, labels in vepoch:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()

                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                    vepoch.set_postfix(val_loss=val_loss / len(val_loader), val_accuracy=100 * correct / total)

        epoch_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * correct / total
        val_losses.append(epoch_val_loss)
        val_accuracies.append(val_accuracy)

        print(f'\nEpoch [{epoch + 1}/{epochs}] - Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}%, '
              f'Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}%')

        scheduler.step(epoch_val_loss)

        # Early stopping
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_epoch = epoch + 1
            patience_counter = 0
            print(f"New best validation accuracy: {val_accuracy:.4f}% at epoch {best_epoch}. Saving model...\n")
            torch.save(model.state_dict(), save_path)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch + 1}")
                break

    print(f"\nTraining complete. Best validation accuracy: {best_val_accuracy:.4f}% at epoch {best_epoch}")
    return train_losses, val_losses, train_accuracies, val_accuracies

train_losses, val_losses, train_accuracies, val_accuracies = train_model(
    alexnet, train_loader, val_loader,
    EPOCHS, criterion, optimizer,
    device, scheduler, patience=PATIENCE,
    save_path=MODEL)

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(train_losses, val_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.legend()
    plt.grid(True)
    plt.show()


plot_losses(train_losses, val_losses)